<a href="https://colab.research.google.com/github/hmyrb/mlp-svm-classification/blob/main/KODLAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Düzenlenmiş RCV1 Veri Seti**

---



In [ ]:
from keras.datasets import reuters
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# ----------------------------
#  Veri hazırlığı
# ----------------------------
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=10000)

word_index = reuters.get_word_index()
index_word = {v + 3: k for k, v in word_index.items()}
index_word[0] = "<PAD>"
index_word[1] = "<START>"
index_word[2] = "<UNK>"

def decode_review(seq):
    return " ".join(index_word.get(i, "?") for i in seq)

x_train_text = [decode_review(x) for x in x_train]
x_test_text  = [decode_review(x) for x in x_test]

CCAT_LIKE_TOPICS = {3, 35, 36, 42, 43, 44, 45}

y_train_bin = np.array([1 if y in CCAT_LIKE_TOPICS else 0 for y in y_train])
y_test_bin  = np.array([1 if y in CCAT_LIKE_TOPICS else 0 for y in y_test])

# ----------------------------
# TF-IDF Vektörleştirme
# ----------------------------
vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1,2),
    min_df=2
)

X_train_tfidf = vectorizer.fit_transform(x_train_text)
X_test_tfidf  = vectorizer.transform(x_test_text)

X_train_dense = X_train_tfidf.toarray()
X_test_dense  = X_test_tfidf.toarray()

# ----------------------------
# MLP İÇİN EKLENEN KISIM (Kritik)
# ----------------------------
# Matris çarpımlarında (N,) hatası almamak için etiketleri (N, 1) yapıyoruz
y_train_rcv1 = y_train_bin.reshape(-1, 1)
y_test_rcv1  = y_test_bin.reshape(-1, 1)

# Değişken isimlerini MLP hücresiyle uyumlu hale getirelim
X_train_rcv1 = X_train_dense
X_test_rcv1  = X_test_dense

print(f"Hazır! RCV1 (Reuters) Train: {X_train_rcv1.shape}, Test: {X_test_rcv1.shape}, Label: {y_train_rcv1.shape}")

**HIGGS Veri Seti**

---



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def prepare_higgs_subset(total_samples=20000, test_size=0.2, random_state=42):
    """
    UCI Repository'den HIGGS veri setini stream ederek okur,
    belirtilen sayıda alt küme oluşturur ve  normalize eder.
    """

    # UCI HIGGS Veri Seti URL'i (Gzip formatında)
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz"

    print(f"1. HIGGS veri seti UCI sunucusundan okunuyor...")
    print("   (Not: Dosyanın tamamı indirilmiyor, sadece gerekli kısım çekiliyor. Lütfen bekleyiniz...)")


    # nrows=total_samples * 1.2 diyerek biraz fazlasını alıyoruz ki karıştırınca (shuffle) denge bozulmasın.
    try:
        # HIGGS veri setinde başlık (header) yoktur.
        # 1. Kolon: Sınıf Etiketi (1.0 = Sinyal, 0.0 = Arkaplan)
        # 2-29. Kolonlar: 28 adet öznitelik (21 düşük seviye, 7 yüksek seviye fiziksel özellik)
        raw_df = pd.read_csv(url, compression='gzip', header=None, nrows=int(total_samples * 1.5))
    except Exception as e:
        print(f"Hata: Veri çekilemedi. İnternet bağlantınızı kontrol edin.\nDetay: {e}")
        return None, None, None, None

    # Veriyi Numpya çevirme
    data = raw_df.to_numpy()

    # X (Öznitelikler) ve y (Etiketler) ayrımı
    # İlk sütun etiket (y), geri kalanı özelliklerdir (X)
    y_raw = data[:, 0].astype(int)
    X_raw = data[:, 1:]

    print(f"   Ham veri çekildi: {X_raw.shape}")

    # 2. ÖRNEKLEM SEÇİMİ (Stratified Subsampling)
    # Veriyi karıştırıp tam istenilen sayıya düşürelim
    print(f"2. {total_samples} adet örneklem seçiliyor ve karıştırılıyor...")

    # Geçici bir split ile veriyi hem karıştırıyoruz hem de boyutunu ayarlıyoruz.
    # stratify=y_raw sayesinde sınıf dengesi (Sinyal/Arkaplan oranı) korunur.
    X_subset, _, y_subset, _ = train_test_split(
        X_raw, y_raw,
        train_size=total_samples,
        stratify=y_raw,
        random_state=random_state
    )

    # 3. NORMALİZASYON (MLP İÇİN ÇOK KRİTİK ADIM)
    # Sigmoid/Tanh gibi aktivasyon fonksiyonlarının doyuma ulaşmaması (vanishing gradient)
    # için verilerin ortalaması 0, varyansı 1 olacak şekilde ölçeklenmesi şarttır.
    print("3. Veri normalize ediliyor (StandardScaler)...")
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_subset)

    # 4. EĞİTİM VE TEST AYRIMI
    print(f"4. Eğitim ve Test setlerine ayrılıyor (%{100*(1-test_size)} - %{100*test_size})...")
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_subset,
        test_size=test_size,
        random_state=random_state,
        stratify=y_subset
    )

    # Boyut düzeltmeleri (Scratch kodlarda vektör hatalarını önlemek için)
    # y vektörünü (N,) yerine (N, 1) yapmak bazen matris çarpımlarında hayat kurtarır.
    # İsteğe bağlıdır, ancak öğrenciler için açık olması iyidir.
    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    print("\n--- HAZIRLIK TAMAMLANDI ---")
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"Sınıf Dağılımı (Train): {np.unique(y_train, return_counts=True)}")
    print("-" * 30)

    return X_train, X_test, y_train, y_test

# --- KULLANIM ---
if __name__ == "__main__":
    X_train, X_test, y_train, y_test = prepare_higgs_subset()

    # ÖĞRENCİLERE VERİLECEK İPUÇLARI:
    print("\n[İPUCU - MLP MODELİ İÇİN]")
    print("1. Giriş Katmanı Boyutu (Input Layer): 28 nöron")
    print("2. Çıkış Katmanı (Output Layer): 1 nöron (Sigmoid aktivasyonu ile 0-1 arası olasılık)")
    print("3. Loss Fonksiyonu: Binary Cross Entropy")
    print("4. Veri StandardScaler ile ölçeklendiği için 'Xavier' veya 'He' initialization kullanmanız önerilir.")

**Aktivasyon Fonksiyonları**

---



In [ ]:
class AktivasyonFonksiyonlari:
    @staticmethod
    def sigmoid(Z):
        return 1 / (1 + np.exp(-np.clip(Z, -15, 15)))

    @staticmethod
    def sigmoid_turevi(Z):
        s = AktivasyonFonksiyonlari.sigmoid(Z)
        return s * (1 - s)

    @staticmethod
    def tanh(Z):
        return np.tanh(Z)

    @staticmethod
    def tanh_turevi(Z):
        return 1 - np.tanh(Z)**2

    @staticmethod
    def relu(Z):
        return np.maximum(0, Z)

    @staticmethod
    def relu_turevi(Z):
        return (Z > 0).astype(float)

    @staticmethod
    def cross_entropy(y_true, y_pred):
        eps = 1e-15
        y_pred = np.clip(y_pred, eps, 1 - eps)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    @staticmethod
    def evaluate(y_true, y_pred_prob):

        y_true = np.array(y_true).flatten()
        y_pred = np.array((y_pred_prob).flatten() > 0.5).astype(int)
        tp = np.sum((y_true == 1) & (y_pred == 1))
        tn = np.sum((y_true == 0) & (y_pred == 0))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))


        acc = (tp + tn) / (tp + tn + fp + fn)
        f1 = (2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0
        cm = np.array([[tn, fp], [fn, tp]])
        return acc, f1, cm

# **MLP (Multi-Layer Perceptron) Algoritması**

---



In [ ]:
class MLP:
  def __init__(self, layer_sizes, aktivasyon_fonksiyonu, learning_rate=0.001):
    self.layer_sizes = layer_sizes
    self.aktivasyon_fonksiyonu = aktivasyon_fonksiyonu
    self.learning_rate = learning_rate
    self.weights =[]
    self.biases = []

    self.v_w, self.v_b = [], [] #momentum ve adam icin
    self.s_w, self.s_b = [], [] #rmsprop ve adam icin
    self.t = 0 #adam icin

#agirlik baslatma
    for i in range(len(layer_sizes)-1):
      limit = np.sqrt(2.0/layer_sizes[i])
      self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i+1])*limit)
      self.biases.append(np.zeros((1, layer_sizes[i+1])))

#optimizer degiskenlerini sifirlama
      self.v_w.append(np.zeros_like(self.weights[-1]))
      self.v_b.append(np.zeros_like(self.biases[-1]))
      self.s_w.append(np.zeros_like(self.weights[-1]))
      self.s_b.append(np.zeros_like(self.biases[-1]))

#forward propagation
  def forward(self, Z):
    self.a = [Z]
    self.z = []

    for i in range(len(self.weights)):
      z = np.dot(self.a[-1], self.weights[i]) + self.biases[i]
      self.z.append(z)

      #aktivasyon secme
      aktivasyon=self.aktivasyon_fonksiyonu[i]
      if aktivasyon == 'sigmoid':
        a = AktivasyonFonksiyonlari.sigmoid(z)
      elif aktivasyon == 'tanh':
        a = AktivasyonFonksiyonlari.tanh(z)
      else: a =  AktivasyonFonksiyonlari.relu(z)
      self.a.append(a)
    return self.a[-1]

#back propagation
  def backward(self,y_true):
    m = y_true.shape[0]
    dz = self.a[-1] - y_true
    dw_list, db_list = [], []

    for i in reversed(range(len(self.weights))):
      dw = np.dot(self.a[i].T, dz)/ m
      db = np.sum(dz, axis=0, keepdims=True) /m
      dw_list.insert(0,dw)
      db_list.insert(0,db)

      if i > 0:
        aktivasyon_turevi = self.aktivasyon_fonksiyonu[i-1]
        if aktivasyon_turevi == 'sigmoid':
          da = AktivasyonFonksiyonlari.sigmoid_turevi(self.z[i-1])
        elif aktivasyon_turevi == 'tanh':
          da = AktivasyonFonksiyonlari.tanh_turevi(self.z[i-1])
        else:
          da = AktivasyonFonksiyonlari.relu_turevi(self.z[i-1])
        dz = np.dot(dz, self.weights[i].T)*da

    return dw_list, db_list

  def optimizerlar(self,dw,db,optimizer,beta1=0.9,beta2=0.999,epsilon=1e-8):
    self.t += 1

    for i in range(len(self.weights)):
      if optimizer == 'sgd':
        self.weights[i] -= self.learning_rate * dw[i]
        self.biases[i] -= self.learning_rate * db[i]

      elif optimizer == 'momentum':
        self.v_w[i] = beta1 * self.v_w[i] + (1 - beta1) * dw[i]
        self.v_b[i] = beta1 * self.v_b[i] + (1 - beta1) * db[i]
        self.weights[i] -= self.learning_rate * self.v_w[i]
        self.biases[i] -= self.learning_rate * self.v_b[i]


      elif optimizer == 'rmsprop':
        self.s_w[i] = beta2*self.s_w[i] + (1-beta2)*(dw[i]**2)
        self.s_b[i] = beta2 * self.s_b[i] + (1 - beta2) * (db[i]**2)
        self.weights[i] -= self.learning_rate * dw[i] / (np.sqrt(self.s_w[i]) + epsilon)
        self.biases[i] -= self.learning_rate * db[i] / (np.sqrt(self.s_b[i]) + epsilon)

      elif optimizer == 'adam':
        self.v_w[i] = beta1*self.v_w[i] + (1-beta1)*dw[i]
        self.v_b[i] = beta1 * self.v_b[i] + (1 - beta1) * db[i]
        self.s_w[i] = beta2 * self.s_w[i] + (1 - beta2) * (dw[i]**2)
        self.s_b[i] = beta2 * self.s_b[i] + (1 - beta2) * (db[i]**2)
        mw_hat = self.v_w[i] / (1 - beta1**self.t)
        mb_hat = self.v_b[i] / (1 - beta1**self.t)
        sw_hat = self.s_w[i] / (1 - beta2**self.t)
        sb_hat = self.s_b[i] / (1 - beta2**self.t)
        self.weights[i] -= self.learning_rate * mw_hat / (np.sqrt(sw_hat) + epsilon)
        self.biases[i] -= self.learning_rate * mb_hat / (np.sqrt(sb_hat) + epsilon)



**MLP Modelini Eğitme**

---



In [ ]:
def modeli_egit(Z, y, layers, aktivasyon_fonksiyonu, optimizer, learning_rate=0.001, epochs=30, batch_size=32):
    model = MLP(layers, aktivasyon_fonksiyonu, learning_rate)
    loss_history = []

    if len(y.shape) == 1:
        y = y.reshape(-1, 1)

    for epoch in range(epochs):
        indices = np.random.permutation(len(Z))
        Z_sh = Z[indices]
        y_sh = y[indices]
        epoch_loss = 0.0

        for i in range(0, len(Z), batch_size):
            batch_Z = Z_sh[i:i+batch_size]
            batch_y = y_sh[i:i+batch_size]

            y_pred = model.forward(batch_Z)
            dw, db = model.backward(batch_y)
            model.optimizerlar(dw, db, optimizer)

            loss = AktivasyonFonksiyonlari.cross_entropy(batch_y, y_pred)
            epoch_loss += loss



        loss_history.append(epoch_loss / (len(Z)/batch_size))
    return model, loss_history

**Performans Metrikleri (HIGGS ve RCV1)**

---


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def run_experiment(X_tr, y_tr, layers, acts, opt='adam', lr=0.001):
    model = MLP(layers, acts, lr)
    loss_history = []
    layers_higgs =[28,64,32,1]
    acts_higgs = ['relu', 'relu', 'sigmoid']

    for epoch in range(30):
        idx = np.random.permutation(len(X_tr))
        X_s, y_s = X_tr[idx], y_tr[idx]
        e_loss = 0
        for i in range(0, len(X_tr), 64):
            xb, yb = X_s[i:i+64], y_s[i:i+64]
            out = model.forward(xb)
            dw, db = model.backward(yb)
            model.optimizerlar(dw, db, opt)
            e_loss += AktivasyonFonksiyonlari.cross_entropy(yb, out)
        loss_history.append(e_loss / (len(X_tr)/64))
    return model, loss_history

m_higgs, _ = run_experiment(X_train, y_train, layers_higgs, acts_higgs, opt='adam')
y_pred = m_higgs.forward(X_test)
acc_h, f1_h, cm_h = AktivasyonFonksiyonlari.evaluate(y_test, y_pred)

m_rcv1, _ = run_experiment(X_train_rcv1, y_train_rcv1, [10000, 128, 64, 1], ['relu', 'relu', 'sigmoid'])
acc_r, f1_r, cm_r = AktivasyonFonksiyonlari.evaluate(y_test_rcv1, m_rcv1.forward(X_test_rcv1))

print("\n---MLP SONUÇLARI ---")
print(f"HIGGS Sonuçlar ->\n Acc: {acc_h:.4f}, F1: {f1_h:.4f}\n")
print(f"RCV1 Sonuçlar ->\n Acc: {acc_r:.4f}, F1: {f1_r:.4f}\n")


fig, ax = plt.subplots(1,2, figsize=(12, 6))

#higgs matrisi
sns.heatmap(cm_h, annot=True, fmt='d', cmap='viridis', ax=ax[0])
ax[0].set_title('Confusion Matrix: HIGGS')
ax[0].set_xlabel('Predicted')
ax[0].set_ylabel('True')

#rcv1 matrisi
sns.heatmap(cm_r, annot=True, fmt='d', cmap='viridis', ax=ax[1])
ax[1].set_title('Confusion Matrix: RCV1')
ax[1].set_xlabel('Predicted')
ax[1].set_ylabel('True')

plt.tight_layout()
plt.show()



**Aktivasyon Fonksiyonu - Optimizer İkilisi Karşılaştırması**

---
*HIGGS ve RCV1 veri setleri için*


In [ ]:
aktivasyon_listesi =['relu','tanh','sigmoid']
optimizer_listesi =['sgd', 'momentum', 'rmsprop', 'adam']
layers_higgs =[28,64,32,1]

sonuclar=[]

for akt in aktivasyon_listesi:
  for opt in optimizer_listesi:
    print(f"Eğitiliyor: Aktivasyon: {akt}, Optimizer: {opt}")

    model, loss_history = run_experiment(X_train, y_train, layers_higgs, akt, opt, lr=0.01)

    last_loss = loss_history[-1]
    print(f"Son Loss: {last_loss}")
    sonuclar.append({'aktivasyon':akt,'optimizer':opt,'loss':last_loss, 'history':loss_history})

In [ ]:
aktivasyon_listesi =['relu','tanh','sigmoid']
optimizer_listesi =['sgd', 'momentum', 'rmsprop', 'adam']
layers_rcv1=[10000, 128,64,1]

sonuclar=[]

for akt in aktivasyon_listesi:
  for opt in optimizer_listesi:
    print(f"Eğitiliyor: Aktivasyon: {akt}, Optimizer: {opt}")

    model, loss_history = run_experiment(X_train_rcv1, y_train_rcv1, layers_rcv1, akt, opt, lr=0.01) # running süresi çok uzun olduğundan ilk 2000 değerle eğitim yapıldı.

    last_loss = loss_history[-1]
    print(f"Son Loss: {last_loss}")
    sonuclar.append({'aktivasyon':akt,'optimizer':opt,'loss':last_loss, 'history':loss_history})

**HIGGS için Optimizer Analizi**

---



In [ ]:
import matplotlib.pyplot as plt

# HIGGS için ayarlar
layers_higgs = [28, 64, 32, 1] # 2 gizli katman örneği [cite: 24]
acts_higgs = ['relu', 'relu', 'sigmoid']
optimizers = ['adam', 'rmsprop', 'momentum']
results_loss = {}

print("Optimizer analizi başlatılıyor...")

for opt in optimizers:
    print(f"Eğitiliyor: {opt}")
    # train_model fonksiyonunu çağırıyoruz
    model, loss_hist = modeli_egit(X_train, y_train, layers_higgs, acts_higgs, optimizer=opt, epochs=30)
    results_loss[opt] = loss_hist

# Grafik Çizimi
plt.figure(figsize=(10, 6))
for opt, losses in results_loss.items():
    plt.plot(range(len(losses)), losses, label=opt)
plt.title('HIGGS Veri Seti: Optimizer Karşılaştırması (Loss vs Epoch)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

**HIGGS için Learning Rate Analizi**

---



In [ ]:
import matplotlib.pyplot as plt

learning_rates = [0.1, 0.01, 0.001]
layers_higgs =[28,64,32,1]
acts_higgs = ['relu', 'relu', 'sigmoid']

plt.figure(figsize=(10, 6))
for lr in learning_rates:
    print(f"Eğitiliyor: lr={lr}")

    _, losses = run_experiment(X_train, y_train, layers_higgs, acts_higgs, opt='adam', lr=lr)
    plt.plot(losses, label=f'LR: {lr}')

plt.title('HIGGS Veri Seti: Learning Rate Değişiminin Yakınsamaya Etkisi')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

**RCV1 için Learning Rate Analizi**

---

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

learning_rates=[0.1,0.01,0.001]
layers_rcv1=[10000, 128,64,1]
acts_rcv1=['relu','tanh','sigmoid']

plt.figure(figsize=(10, 6))
for lr in learning_rates:
    print(f"Eğitiliyor: lr={lr}")

    _,losses =run_experiment(X_train_rcv1, y_train_rcv1, layers_rcv1, acts_rcv1, opt='adam', lr=lr)
    plt.plot(losses, label=f'LR: {lr}')

plt.title('RCV1 Veri Seti: Learning Rate Değişiminin Yakınsamaya Etkisi')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

**Aktivasyon Fonksiyonu Karşılaştırma**

---



In [ ]:
import matplotlib.pyplot as plt

def aktivasyon_karsilastirma(X_tr, y_tr, input_size, veri_adi):
    # Dinamik katman yapısına uygun aktivasyon listeleri
    denenecek_fonksiyonlar = {
        'ReLU': ['relu', 'relu', 'sigmoid'],
        'Sigmoid': ['sigmoid', 'sigmoid', 'sigmoid']
    }

    plt.figure(figsize=(10, 6))

    for isim, akt_listesi in denenecek_fonksiyonlar.items():
        print(f"{veri_adi} eğitiliyor: {isim}")


        layers = [input_size, 64, 32, 1]

        #  eğitim
        _, losses = run_experiment(X_tr, y_tr, layers, akt_listesi, opt='adam')

        # losses listesi boş değilse çizdir
        if losses:
            plt.plot(losses, label=isim)

    plt.title(f'{veri_adi}: ReLU vs Sigmoid Yakınsama Analizi')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

# HIGGS için analiz (28 özellik)
aktivasyon_karsilastirma(X_train, y_train, 28, 'HIGGS')

# RCV1 için analiz (10000 özellik)
aktivasyon_karsilastirma(X_train_rcv1, y_train_rcv1, 10000, 'RCV1')

# **SVM - PEGASOS**

In [ ]:
import numpy as np
from scipy import sparse

class PegasosSVM:
    def __init__(self, lambda_reg=0.01, iterations=1000):
      self.lambda_reg = lambda_reg
      self.iterations = iterations
      self.w = None
      self.b = 0

    def fit(self, X, y):
      y_svm = np.where(y<=0, -1 ,1).flatten()

      n_samples, n_features =X.shape
      self.w = np.zeros(n_features)

      for t in range(1, self.iterations + 1):
        i = np.random.randint(0, n_samples)
        x_i = X[i]
        y_i = y_svm[i]

        eta = 1 / (self.lambda_reg * t)

        if sparse.issparse(x_i):
          decision = y_i * x_i.dot(self.w)[0]
        else:
          decision = y_i * np.dot(x_i, self.w)
       #hinge loss, l2 ile
        if decision < 1:
          self.w = (1 - eta * self.lambda_reg) * self.w + (eta * y_i * x_i)
        else:
          self.w = (1- eta * self.lambda_reg) * self.w

        #normu sifirla
        norm_w = np.linalg.norm(self.w)
        if norm_w >0:
          projection = min(1, (1/np.sqrt(self.lambda_reg))/norm_w)
          self.w = self.w * projection

    def predict(self,X):
      if sparse.issparse(X):
        score = X.dot(self.w)+self.b
      else:
        score = np.dot(X, self.w) + self.b
      return np.where(score >= 0, 1, 0)


**Algoritma Eğitimi** ve Veri setleri için **Performans Metrikleri**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. HIGGS Üzerinde Eğitim
print("SVM (Pegasos) HIGGS üzerinde eğitiliyor...")
svm_higgs = PegasosSVM(lambda_reg=0.01, iterations=1000)
svm_higgs.fit(X_train, y_train)

y_pred_higgs = svm_higgs.predict(X_test)
acc_h, f1_h, cm_h = AktivasyonFonksiyonlari.evaluate(y_test, y_pred_higgs)

# 2. RCV1 Üzerinde Eğitim
print("SVM (Pegasos) RCV1 üzerinde eğitiliyor...")
# RCV1 yüksek boyutlu olduğu için lambda_reg değerini biraz daha küçük tutabiliriz
svm_rcv1 = PegasosSVM(lambda_reg=0.0001, iterations=1000)
svm_rcv1.fit(X_train_rcv1, y_train_rcv1)

y_pred_rcv1 = svm_rcv1.predict(X_test_rcv1)
acc_r, f1_r, cm_r = AktivasyonFonksiyonlari.evaluate(y_test_rcv1, y_pred_rcv1)

print("\n--- SVM (PEGASOS) SONUÇLARI ---")
print(f"HIGGS -> Acc: {acc_h:.4f}, F1: {f1_h:.4f}")
print(f"RCV1  -> Acc: {acc_r:.4f}, F1: {f1_r:.4f}")

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

#higgs matrisi
sns.heatmap(cm_h, annot=True, fmt='d', cmap='Blues', ax=ax[0])
ax[0].set_title('SVM Confusion Matrix: HIGGS')
ax[0].set_xlabel('Predicted')
ax[0].set_ylabel('True')

#rcv1 matrisi
sns.heatmap(cm_r, annot=True, fmt='d', cmap='Blues', ax=ax[1])
ax[1].set_title('SVM Confusion Matrix: RCV1')
ax[1].set_xlabel('Predicted')
ax[1].set_ylabel('True')

plt.tight_layout()
plt.show()

**Hibrit Model (Ensemble Learning)**

---



In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def hibrit_model(mlp_model, svm_model, X_train_sub, y_train_sub, X_test_sub, y_test_sub, veri_adi):
    print(f"\n {veri_adi} için eğitiliyor")

    #meta-feature oluşturma
    mlp_train_meta = mlp_model.forward(X_train_sub).flatten()

    #svm karar mesafesi
    if sparse.issparse(X_train_sub):
        svm_train_meta = X_train_sub.dot(svm_model.w).flatten()
    else:
        svm_train_meta = np.dot(X_train_sub, svm_model.w).flatten()


    #[MLP_prob, SVM_Score]
    X_meta_train = np.column_stack((mlp_train_meta, svm_train_meta))
    y_meta_train = y_train_sub.flatten()

    #Meta Model ve Logistic Regression
    meta_model = LogisticRegression(max_iter=1000)
    meta_model.fit(X_meta_train, y_meta_train)

    mlp_test_meta = mlp_model.forward(X_test_sub).flatten()

    if sparse.issparse(X_test_sub):
        svm_test_meta = X_test_sub.dot(svm_model.w).flatten()
    else:
        svm_test_meta = np.dot(X_test_sub, svm_model.w)

    X_meta_test = np.column_stack((mlp_test_meta, svm_test_meta))
    y_pred = meta_model.predict(X_meta_test)

    acc, f1, cm = AktivasyonFonksiyonlari.evaluate(y_test_sub, y_pred)
    print(f"{veri_adi} -> Acc: {acc:.4f}, F1: {f1:.4f}")

    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    plt.title(f'Hibrit Confusion Matrix: {veri_adi}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    return acc, f1

**Hibrit Model İçin Performans Metrikleri**

---



In [ ]:
# HIGGS için hibrit modelde performans metrikleri
acc_h_h, f1_h_h= hibrit_model(m_higgs, svm_higgs, X_train, y_train, X_test, y_test, "HIGGS")

# RCV1 için hibrit modelde performans metrikleri
acc_r_h, f1_r_h = hibrit_model(m_rcv1, svm_rcv1, X_train_rcv1, y_train_rcv1, X_test_rcv1, y_test_rcv1, "RCV1")

